In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
                      
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn import linear_model
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

import nltk
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import Normalizer


from sklearn.model_selection import RandomizedSearchCV
import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# path = r"/Users/charlotte/Desktop/BT4222/train_labelled_final.csv"
# reviews = pd.read_csv(path)
reviews = pd.read_csv('./train_cleaned.csv', encoding= 'unicode_escape')

In [3]:
reviews.head()

,Unnamed: 0,Restaurant,Review,Label,Stars,Date,Review_clean
0,0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00,fresh ingredients friendly peeps and so much c...
1,1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00,a small selection of italian wines by the glas...
2,2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00,the plus point is that the price remains the s...
3,3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00,same for more i ordered what i thought was the...
4,4,Brawn & Brains Coffee,I don't get why it can't just be full service....,1,3,2019-12-17T00:00:00,i don t get why it can t just be full service ...


In [4]:
reviews.Label.value_counts()

2    1117
3     699
5     693
1     520
4     434
Name: Label, dtype: int64

In [5]:
reviews.Review_clean = reviews.Review_clean.apply(lambda x: np.str_(x))

In [6]:
x = reviews.Review_clean
y = reviews.Label

In [7]:
# split x and y into training and testing sets
# stratify returns training and test subsets that have the same proportions of class labels as the input dataset.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, stratify = y)

In [8]:
# examine the object shapes
print(y_train.value_counts())
print(y_test.value_counts())

2    838
3    524
5    520
1    390
4    325
Name: Label, dtype: int64
2    279
3    175
5    173
1    130
4    109
Name: Label, dtype: int64


In [9]:
## Building the vocabulary

In [10]:
vect = CountVectorizer()

In [11]:
vect.fit(x_train)
x_train_dtm = vect.transform(x_train)

# only transform x_test
x_test_dtm = vect.transform(x_test)

In [12]:
# examine the shapes: rows are documents, columns are terms (aka "tokens" or "features")
print(x_train_dtm.shape)
print(x_test_dtm.shape)

(2597, 5135)
(866, 5135)


In [13]:
# examine the last 50 features
print(vect.get_feature_names()[-50:])

['xlbs', 'xo', 'ya', 'yahoo', 'yakitori', 'yam', 'yamazaki', 'yang', 'yankee', 'yarra', 'yay', 'year', 'years', 'yeh', 'yelling', 'yellow', 'yelp', 'yelper', 'yelpers', 'yeoh', 'yes', 'yesterday', 'yet', 'yielding', 'yo', 'yoghurt', 'yogurt', 'yolk', 'york', 'you', 'young', 'your', 'yours', 'yourself', 'yu', 'yuan', 'yucatan', 'yuen', 'yummier', 'yumminess', 'yummmm', 'yummmmmmyyyy', 'yummmy', 'yummy', 'yusheng', 'yuzu', 'zealand', 'zhajing', 'zhi', 'ziplock']


# Comparing the accuracy of different approaches

In [14]:
vects = [CountVectorizer(),TfidfVectorizer()]
vectnames = ["Count Vect","Tfidf Vect"]


clfs = [BaggingClassifier(RandomForestClassifier()), MultinomialNB(),LinearSVC(),LogisticRegression(),RandomForestClassifier(),linear_model.SGDClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]
clfnames = ["bagging random forest", "Naive Bayes","Linear SVM","Logistic Regression","Random Forest","Stochastic Gradient Descent", "Gradient Boosting", "Ada Boost"]

In [15]:
#building a pipeline

for vectname, vect, in zip(vectnames, vects):

    for clfname, clf in zip(clfnames, clfs):
        pipe = Pipeline([
            ('vect', vect),
            ('clf', clf),
        ]
        )       

        pipe.fit(x_train, y_train)
        pred = pipe.predict(x_test)
        train_acc = metrics.accuracy_score(y_train, pipe.predict(x_train))
        test_acc = metrics.accuracy_score(y_test, pred)
        #roc_auc = metrics.roc_auc_score(y_test, pred)
        print("{} + {} - train acc: {} test acc: {} ".format(vectname, clfname, train_acc, test_acc))
        #scores = cross_val_score(pipe, x, y,scoring='roc_auc')
        #print(scores.mean())

    

Count Vect + bagging random forest - train acc: 0.9888332691567193 test acc: 0.6836027713625866 
Count Vect + Naive Bayes - train acc: 0.9083557951482479 test acc: 0.7413394919168591 
Count Vect + Linear SVM - train acc: 0.9984597612629957 test acc: 0.7551963048498845 


c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Count Vect + Logistic Regression - train acc: 0.9907585675779745 test acc: 0.7494226327944573 
Count Vect + Random Forest - train acc: 1.0 test acc: 0.710161662817552 
Count Vect + Stochastic Gradient Descent - train acc: 0.9973045822102425 test acc: 0.7575057736720554 
Count Vect + Gradient Boosting - train acc: 0.8475163650365807 test acc: 0.7297921478060047 
Count Vect + Ada Boost - train acc: 0.6372737774355025 test acc: 0.6177829099307159 
Tfidf Vect + bagging random forest - train acc: 0.9865229110512129 test acc: 0.6755196304849884 
Tfidf Vect + Naive Bayes - train acc: 0.797458606083943 test acc: 0.6327944572748267 
Tfidf Vect + Linear SVM - train acc: 0.9938390450519831 test acc: 0.7759815242494227 
Tfidf Vect + Logistic Regression - train acc: 0.9056603773584906 test acc: 0.7551963048498845 
Tfidf Vect + Random Forest - train acc: 1.0 test acc: 0.7055427251732102 
Tfidf Vect + Stochastic Gradient Descent - train acc: 0.9946091644204852 test acc: 0.7655889145496536 
Tfidf Vect

In [16]:
# doing randomised search 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])


parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__analyzer':['char', 'word', 'char_wb'],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__binary': [True, False],
    'tfidf__norm': [None, 'l1', 'l2'], 
    'lr__dual': [True,False],
    'lr__max_iter': [100,200, 500],
    'lr__C' :np.logspace(0, 4, num=10)
  
    
}



random =RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=2)

start_time = time.time()
random_result = random.fit(x_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

pred = random.predict(x_test)
test_acc = metrics.accuracy_score(y_test, pred)
#roc_auc = metrics.roc_auc_score(y_test, pred)
pre_macro = metrics.precision_score(y_test, pred, average="macro")
recall_macro = metrics.recall_score(y_test, pred, average="macro")
f1_macro = metrics.f1_score(y_test, pred, average="macro")


print("test acc: {} recall: {} f1: {}".format(test_acc, recall_macro, f1_macro))


Best: 0.718129 using {'tfidf__norm': 'l1', 'tfidf__ngram_range': (1, 2), 'tfidf__max_df': 0.5, 'tfidf__binary': False, 'tfidf__analyzer': 'word', 'lr__max_iter': 200, 'lr__dual': False, 'lr__C': 1291.5496650148827}
Execution time: 68.94967269897461 ms
test acc: 0.7494226327944573 recall: 0.730016618021141 f1: 0.7316777338925754


c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
vect = TfidfVectorizer(ngram_range=(1, 2), max_df=0.75, norm= 'l2', binary = True, analyzer = 'word')
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)


lr = LogisticRegression(max_iter = 100, dual = False, C = 10000)
lr.fit(x_train_dtm, y_train)
y_pred_class = lr.predict(x_test_dtm)

# Get the training accuracy
print('Training Accuracy: ', metrics.accuracy_score(y_train, lr.predict(x_train_dtm)))
# print the accuracy of its predictions
print('Test Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))


Training Accuracy:  1.0
Test Accuracy:  0.7736720554272517


c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC



vect = TfidfVectorizer(ngram_range=(1, 2), max_df=0.75, norm= 'l2', binary = True, analyzer = 'word')
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)


lr = OneVsRestClassifier(SVC())
lr.fit(x_train_dtm, y_train)
y_pred_class = lr.predict(x_test_dtm)

# Get the training accuracy
print('Training Accuracy: ', metrics.accuracy_score(y_train, lr.predict(x_train_dtm)))
# print the accuracy of its predictions
print('Test Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

Training Accuracy:  0.9992298806314979
Test Accuracy:  0.7321016166281755


In [19]:
## RNN

In [20]:
# Import necessary libraries
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.models import Model
from keras.models import Sequential

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
reviews.Review_clean=reviews.Review_clean.astype(str)

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
#tokenizer.fit_on_texts(reviews.Review_clean.values)

tokenizer.fit_on_texts(reviews.Review_clean.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Truncate and pad the input sequences so that they are all in the same length for modeling

X = tokenizer.texts_to_sequences(reviews.Review_clean.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
Y = pd.get_dummies(reviews.Label).values
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, BatchNormalization



model = Sequential()

model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))


model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


# Loss: 0.785
#   Accuracy: 0.734

In [ ]:
accr

In [ ]:

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
new_complaint = ['get full service menu decent prices pretty expensive']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Price', 'Taste', 'Service', 'Ambience', 'Others']
print(pred, labels[np.argmax(pred)])

In [ ]:
df = pd.DataFrame(x_test)
df.head()

In [ ]:
ans = []

for i in df.Review_clean:
    new_complaint = [i]
    
    seq = tokenizer.texts_to_sequences(new_complaint)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    labels = ['1', '2', '3', '4', '5']
    
    #print(pred, labels[np.argmax(pred)])
    #print(labels[np.argmax(pred)])
    ans.append(labels[np.argmax(pred)])
    

In [ ]:
df['Predicted'] = ans
df['Actual']=y_test

In [ ]:
df.head()

In [ ]:
confusion_matrix = pd.crosstab(df['Actual'], df['Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)